In [1]:
import warnings
warnings.filterwarnings('ignore')

path = "../../kaggle/data/"

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

### I. Chargement des données

#### 1. Train & Test

In [2]:
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")

In [3]:
print('-' * 50)
display('*** Train ***')
print()
display(train.head(2))
display(train.tail(2))
print()
print('-' * 50)
print()
display('***Test***')
print()
display(test.head(2))
display(test.tail(2))
print('-' * 50)

--------------------------------------------------


'*** Train ***'

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0


,id,date,store_nbr,family,sales,onpromotion
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.0,8
3000887,3000887,2017-08-15,9,SEAFOOD,16.0,0



--------------------------------------------------



'***Test***'

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0


,id,date,store_nbr,family,onpromotion
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9
28511,3029399,2017-08-31,9,SEAFOOD,0


--------------------------------------------------


#### 2. Descriptions des variables des train & test dataset: 

- **date** : The date when the data was recorded.
- **store_nbr** : Identifies the store at which the products are sold.
- **family** : Identifies the type of product sold.
- **onpromotion** : Gives the total number of items in a product family that were being promoted at a store at a given date.

 The <ins>**Target variable**</ins>
- **sales** : Gives the total sales for a product family at a particular store at a given date. Fractional values are possible.
  

In [4]:
display('*** Train ***')
display(len(train.store_nbr.unique()))
display(len(train.family.unique()))
print('-' * 50)
display('***Test***')
display(len(test.store_nbr.unique()))
display(len(test.family.unique()))

'*** Train ***'

54

33

--------------------------------------------------


'***Test***'

54

33

In [5]:
# liste des stores
train.store_nbr.unique()

array([ 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,  2, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29,  3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,  4,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49,  5, 50, 51, 52, 53, 54,  6,
        7,  8,  9])

In [6]:
# liste de famille de produits vendus par chque magasin
train.family.unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

transactions - The total number of transactions that occurred in a store on a given date.
From oil.csv:
date - The date when the oil price was recorded.
dcoilwtico - The daily oil price; "WTI" refers to West Texas Intermediate, a grade of crude oil used as a benchmark in oil pricing.
From holidays_events.csv:
date - The date of the holiday or event.
type - The type of holiday or event (e.g., Holiday, Event, Bridge, Work Day, Transfer).
locale - The locale where the holiday is observed (e.g., National, Regional, Local).
locale_name - The name of the locale where the holiday is observed.
description - A brief description of the holiday or event.
transferred - Indicates whether the holiday was transferred to another date (True or False).
From stores.csv:
store_nbr - The store number, a unique identifier for each store.
city - The city where the store is located.
state - The state where the store is located.
type - The type of store.
cluster - A grouping of similar stores.

In [ ]:
holiday_events = pd.read_csv(path + "holidays_events.csv")
oil = pd.read_csv(path + "oil.csv")
stores = pd.read_csv(path + "stores.csv")

transactions = pd.read_csv(path + "transactions.csv")

train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
holiday_events["date"] = pd.to_datetime(holiday_events.date)
oil["date"] = pd.to_datetime(oil.date)
transactions["date"] = pd.to_datetime(transactions.date)